# Interprétabilité des modèles
## 0. Overview

Les enjeux derrière l'interprétabilité des modèles, ses motivations ainsi que des méthodes pour les appliquer ont été présentées par **Maxence Brochard**. Vous pouvez à tout moment retrouver les ressources ici : https://lion.app.box.com/folder/65170147483 

=> **L'objectif de ce notebook est la mise en pratique de l'interprétabilité des modèles sur des cas concrets**

### 0.1 Données

Pour la suite de ce notebook, nous nous baserons sur un jeu de données très simple, qui vise à déterminer le prix de vente des maisons de Boston en fonction de différents indicateurs.

In [ ]:
from sklearn.datasets import load_boston
import pandas as pd

boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.DataFrame(boston.target, columns=["Houses prices"])
print(boston.DESCR)

## 1. Focus sur les Random Forests

Pour commencer, vous devez installer le package **treeinterpreter** : `pip install treeinterpreter` dans votre CLI.

### 1.1 Import des packages

In [ ]:
import warnings # On enlève les warnings pour la suite du notebook
warnings.filterwarnings('ignore')

In [ ]:
from treeinterpreter import treeinterpreter as ti
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.grid_search import GridSearchCV

### 1.2 Idée générale

Nous allons essayer de nous intéresser un peu plus en profondeur aux Random Forests, et plus particulièrement aux prédictions obtenus par ces modèles, en cherchant à décomposer une prédiction, par exemple pour une maison *i* comme la somme des contributions de chaque variable pour cette maison, i.e. : 
<center> $ prediction^i=biais+contributionVariable_1^i+…+contributionVariable_n^i $ </center> 

Très peu de packages proposent actuellement de rentrer dans ce niveau de détails à l'heure actuelle. Nous nous baserons sur **treeinterpreter**. Ce dernier propose la décomposition exposée au dessus pour différents modèles existants sous *scikit-learn*, tels que :
* DecisionTreeRegressor
* DecisionTreeClassifier
* ExtraTreeRegressor
* ExtraTreeClassifier
* RandomForestRegressor
* RandomForestClassifier
* ExtraTreesRegressor
* ExtraTreesClassifier




### 1.3 Première Random Forest
**Exercice :**
Pour commencer, séparez les données à disposition en 2 échantillons (via la fonction *train_test_split*) en définissant une graîne aléatoire (=1234) :
* un échantillon d'apprentissage (2/3 des données)
* un échantillon de validation (1/3 des données)

Entraînez ensuite une Random Forest (les paramètres par défaut suffiront pour l'exemple) sur la base de vos données d'apprentissage, en indiquant une graine aléatoire (=1234) pour figer les résultats.

*NB : Nous noterons X_train, X_test, y_train, y_test nos échantillons d'apprentissages et de validation, et rf le modèle entraîné*

Nous allons maintenant choisir 2 points de données de notre échantillon de test, sur lequel nous allons prédire notre cible

In [ ]:
tworows = X_test.iloc[:2,]
for i,prediction in enumerate(rf.predict(tworows)):
    print("Prédiction pour ligne {} de notre échantillon de test : {}".format(i,round(prediction,2)))

On constate que les prédictions sont très éloignées pour ces 2 points de données. L'idée est donc de comprendre maintenant quelles sont les variables qui ont le plus contribuées (aussi bien négativement que positivement) les prédictions.

Nous allons donc utiliser le package **treeinterpreter**.

La structure est relativement simple, et nous permet, sur la base d'un modèle déjà entraîné, de récupérer pour des points de données de l'échantillon de test, la prédiction du modèle, le biais, ainsi que les contributions de chaque variable :

In [ ]:
prediction, biais, contributions = ti.predict(rf, tworows)

Si l'on commence par s'intéresser au contenu de *prediction*, on remarque bien qu'on récupère les mêmes valeurs que ci-dessus. (à ceci près qu'elles sont arrondies à deux décimales)

In [ ]:
prediction

Le contenu de la variable *biais*, qui n'est rien de plus que la moyenne sur l'échantillon d'apprentissage, est logiquement le même qu'importe le point de données de test.

In [ ]:
biais

Enfin, la variable *contributions* contient deux arrays de dimensions 1x13 chacun, représentant pour chaque prédiction, la contribution de chacune des variables.

In [ ]:
contributions

On a donc tout ce qui est nécessaire pour déterminer les contributions de chacune des variables aux deux prédictions, en rappelant que :
<center> $ prediction=biais+contributionVariable1+…+contributionVariablen $ </center> 

In [ ]:
for i in range(len(tworows)):
    print("Point de données {}".format(i))
    print("Biais {}".format(biais[i]))
    print("Contributions des variables (par décroissante absolue) :")
    for c, feature in sorted(zip(contributions[i], 
                                 boston.feature_names), 
                             key=lambda x: -abs(x[0])):
        print("{} : {}".format(feature, round(c, 2)))
    print("-"*20) 

**Exercice** : A partir des informations récupérées (biais et contributions) grâce au module *.predict* du package **treeinterpreter**, recalculer les prédictions pour nos 2 points de données de test, et vérifier qu'elles correspondent bien à ce que nous avions obtenu avec *scikit-learn*.

Ce package est donc très pratique pour pouvoir mieux interpréter les prédictions de nos modèles de Random Forests pour certains points de données.

On y trouve **2 intérêts majeurs** :
* Comprendre pourquoi les valeurs prédites sur 2 jeux de données sont différentes, et quelles sont les variables en causes. Sur notre exemple, on pourrait par exemple chercher à comprendre d'où viennent les différences de prix des maisons de plusieurs voisinages
* Débugger un modèle et/ou les données, en cherchant par exemple à comprendre pourquoi les valeurs prédites sur un nouveau jeu de données ne matchent pas avec celle d'anciennes données

=> **Essayons de développer le premier cas**.

**Exercice** : Splitter le jeu de données de test *(X_test)* en 2 sous échantillons (respectivement *ech1* & *ech2*) de tailles égales. En utilisant la Random Forest déjà entraînée, calculez et stockez les prédictions associées à ces 2 sous échantillons. Calculez en ensuite la moyenne.

On peut constater que les prédictions moyennes sont relativement différentes sur les 2 échantillons.

**Exerice** : Appliquer la décomposition vue au dessus sur les 2 sous échantillons *ech1* et *ech2*. On notera *prediction1, biais1* et *contributions1* (respectivement *2*) les variables dans lequelles seront stockées les résultats.

Sommez ensuite les contributions par variable pour chaque sous échantillon dans deux variables, respectivement *totalc1* et *totalc2*.

Dans la mesure où les biais sont les mêmes (puisque calculés sur le même échantillon d'apprentissage), la différence entre les prédictions moyennes sur les 2 sous échantillons provient uniquement des contributions des différentes variables.
Dans d'autres termes, la somme des contributions des variables est égale à la différence entre les prédictions moyennes. Ce que nous pouvons facilement vérifier.


In [ ]:
np.sum(totalc1 - totalc2),np.mean(prediction1) - np.mean(prediction2)

**Exercice** : Calculer les différences de contributions de chaque variable entre les 2 sous échantillons, et appuyez vous sur le dictionnaire des données disponible en début de notebook pour interpréter.

# Crédits
Nous allons baser la suite sur le lien suivant : http://blog.datadive.net/random-forest-interpretation-with-scikit-learn/